# PromptTemplate과 Model 분리 실행하기

이 노트북에서는 **PromptTemplate**과 **Model**을 분리하여 각각 실행하는 방법을 알아봅니다.

## LCEL 체인 vs 분리 실행 비교

| 방식 | 코드 | 특징 |
|------|------|------|
| **LCEL 체인** | `chain = template \| model` | 한 번에 실행, 코드 간결 |
| **분리 실행** | `template.invoke()` → `model.invoke()` | 중간 결과 확인 가능, 디버깅 용이 |

## 언제 분리 실행을 사용할까?

- **디버깅**: 프롬프트가 올바르게 생성되었는지 확인할 때
- **프롬프트 재사용**: 같은 프롬프트를 여러 모델에 적용할 때
- **로깅**: 생성된 프롬프트를 저장하거나 기록할 때
- **조건부 처리**: 프롬프트 결과에 따라 다른 모델을 선택할 때

---

# 1. Ollama 설치 및 서버 실행

In [ ]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [ ]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. PromptTemplate과 Model 분리 실행

**실행 흐름:**

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  입력 변수       │ ──▶ │  PromptTemplate │ ──▶ │  완성된 프롬프트  │
│  {context,      │     │  .invoke()      │     │  (StringPrompt) │
│   question}     │     │                 │     │                 │
└─────────────────┘     └─────────────────┘     └─────────────────┘
                                                        │
                                                        ▼
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  최종 응답       │ ◀── │     Model       │ ◀── │  완성된 프롬프트  │
│  (completion)   │     │    .invoke()    │     │                 │
└─────────────────┘     └─────────────────┘     └─────────────────┘
```

**코드 설명:**

### Step 1: PromptTemplate 생성
- `{context}`, `{question}` 변수를 포함한 템플릿 정의
- 재사용 가능한 프롬프트 구조

### Step 2: template.invoke() - 프롬프트 생성
- 변수에 실제 값을 대입하여 완성된 프롬프트 생성
- 이 단계에서 **중간 결과를 확인**할 수 있음

### Step 3: model.invoke() - LLM 호출
- 완성된 프롬프트를 모델에 전달하여 응답 생성

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate

# Step 1: PromptTemplate 생성 (재사용 가능)
template = PromptTemplate.from_template('''아래 작성한 컨텍스트(Context)를 기반으로
    질문(Question)에 대답하세요. 제공된 정보로 대답할 수 없는 질문이라면 "모르겠어요." 라고 답하세요.

Context: {context}

Question: {question}

Answer: ''')

# Step 2: Model 생성 (재사용 가능)
model = OllamaLLM(model='llama3.2')

# 4. 분리 실행 - 중간 결과 확인

**핵심 포인트:**
- `template.invoke()` 결과를 변수에 저장하여 확인 가능
- 프롬프트가 의도대로 생성되었는지 디버깅할 때 유용

In [ ]:
# Step 3: template.invoke() - 프롬프트 생성 (중간 결과)
prompt = template.invoke({
    'context': '''거대 언어 모델(LLM)은 자연어 처리(NLP) 분야의 최신 발전을 이끌고 있습니다.
    거대 언어 모델은 더 작은 모델보다 우수한 성능을 보이며, NLP 기능을 갖춘 애플리케이션을 개발하는 개발자들에게
    매우 중요한 도구가 되었습니다. 개발자들은 Hugging Face의 `transformers` 라이브러리를
    활용하거나, `openai` 및 `cohere` 라이브러리를 통해 OpenAI와 Cohere의 서비스를 이용하여
    거대 언어 모델을 활용할 수 있습니다.
    ''',
    'question': '거대 언어 모델은 어디서 제공하나요?'
})

# 중간 결과 확인 (디버깅용)
print("=== 생성된 프롬프트 ===")
print(prompt.text)
print("=" * 50)

In [ ]:
# Step 4: model.invoke() - LLM 호출
completion = model.invoke(prompt)

print("=== LLM 응답 ===")
print(completion)

---

## 코드 비교: 분리 실행 vs LCEL 체인

### 분리 실행 (이 노트북)
```python
prompt = template.invoke({'context': '...', 'question': '...'})
completion = model.invoke(prompt)
```

### LCEL 체인 (04.prompt.ipynb)
```python
chain = template | model
response = chain.invoke({'context': '...', 'question': '...'})
```

## OpenAI에서 Ollama로 변경 시

```python
# 원본 (OpenAI)
from langchain_openai.llms import OpenAI
model = OpenAI()

# 변경 (Ollama)
from langchain_ollama import OllamaLLM
model = OllamaLLM(model='llama3.2')
```

> **참고**: `OllamaLLM`은 텍스트 완성 모델이고, `ChatOllama`는 대화형 모델입니다. PromptTemplate과 함께 사용할 때는 `OllamaLLM`이 적합합니다.

## 활용 팁

1. **개발 단계**: 분리 실행으로 프롬프트 확인 → 완성 후 LCEL 체인으로 리팩토링
2. **프롬프트 로깅**: `prompt.text`를 파일이나 DB에 저장하여 추후 분석
3. **A/B 테스트**: 같은 프롬프트를 여러 모델에 적용하여 성능 비교